In [3]:

import os
import boto3

from dotenv import load_dotenv

# Load from .env file (create a .env file with your credentials)
load_dotenv()

# Pull the credentials
aws_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
aws_session_token = os.environ.get('AWS_SESSION_TOKEN')

print(f"AWS_ACCESS_KEY_ID: {'✓ Set' if aws_access_key else '✗ Not set'}")
print(f"AWS_SECRET_ACCESS_KEY: {'✓ Set' if aws_secret_key else '✗ Not set'}")
print(f"AWS_SESSION_TOKEN: {'✓ Set' if aws_session_token else '✗ Not set'}")

AWS_ACCESS_KEY_ID: ✓ Set
AWS_SECRET_ACCESS_KEY: ✓ Set
AWS_SESSION_TOKEN: ✓ Set


In [5]:
# List all EC2 instances and their security groups
ec2_client = boto3.client('ec2', region_name='us-west-1',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token
)

response = ec2_client.describe_instances()

print("EC2 Instances and their Security Groups:")
print("=" * 60)

for reservation in response['Reservations']:
    for instance in reservation['Instances']:
        instance_id = instance['InstanceId']
        instance_name = 'N/A'
        
        # Get instance name from tags
        if 'Tags' in instance:
            for tag in instance['Tags']:
                if tag['Key'] == 'Name':
                    instance_name = tag['Value']
        
        print(f"\nInstance: {instance_id} ({instance_name})")
        print(f"State: {instance['State']['Name']}")
        
        # List security groups for this instance
        if 'SecurityGroups' in instance:
            print(f"Security Groups ({len(instance['SecurityGroups'])}):")
            for sg in instance['SecurityGroups']:
                print(f"  - {sg['GroupName']} ({sg['GroupId']})")


EC2 Instances and their Security Groups:

Instance: i-0fd1bc9c9a1133708 (CloudWorkstation)
State: running
Security Groups (2):
  - SSMVPCEndpoints (sg-00c62ab97c95dd9c0)
  - RemoteAccess (sg-0fff8a116b3c1900a)


In [6]:
# Get detailed security group information including all rules
# First, collect all unique security group IDs from instances
security_group_ids = set()

for reservation in response['Reservations']:
    for instance in reservation['Instances']:
        if 'SecurityGroups' in instance:
            for sg in instance['SecurityGroups']:
                security_group_ids.add(sg['GroupId'])

# Now get detailed information for all security groups
if security_group_ids:
    sg_response = ec2_client.describe_security_groups(GroupIds=list(security_group_ids))
    
    print("\n\nDetailed Security Group Rules:")
    print("=" * 80)
    
    for sg in sg_response['SecurityGroups']:
        print(f"\n🔒 Security Group: {sg['GroupName']} ({sg['GroupId']})")
        print(f"   Description: {sg['Description']}")
        print(f"   VPC: {sg.get('VpcId', 'N/A')}")
        
        # Inbound Rules (Ingress)
        print(f"\n   📥 INBOUND RULES ({len(sg['IpPermissions'])}):")
        if sg['IpPermissions']:
            for rule in sg['IpPermissions']:
                protocol = rule.get('IpProtocol', 'N/A')
                from_port = rule.get('FromPort', 'All')
                to_port = rule.get('ToPort', 'All')
                
                print(f"      Protocol: {protocol} | Ports: {from_port}-{to_port}")
                
                # IP ranges
                for ip_range in rule.get('IpRanges', []):
                    cidr = ip_range.get('CidrIp', 'N/A')
                    desc = ip_range.get('Description', '')
                    print(f"        → {cidr} {desc}")
                
                # IPv6 ranges
                for ipv6_range in rule.get('Ipv6Ranges', []):
                    cidr = ipv6_range.get('CidrIpv6', 'N/A')
                    desc = ipv6_range.get('Description', '')
                    print(f"        → {cidr} {desc}")
                
                # Security group references
                for sg_ref in rule.get('UserIdGroupPairs', []):
                    ref_sg_id = sg_ref.get('GroupId', 'N/A')
                    desc = sg_ref.get('Description', '')
                    print(f"        → SG: {ref_sg_id} {desc}")
        else:
            print("      (No inbound rules)")
        
        # Outbound Rules (Egress)
        print(f"\n   📤 OUTBOUND RULES ({len(sg['IpPermissionsEgress'])}):")
        if sg['IpPermissionsEgress']:
            for rule in sg['IpPermissionsEgress']:
                protocol = rule.get('IpProtocol', 'N/A')
                from_port = rule.get('FromPort', 'All')
                to_port = rule.get('ToPort', 'All')
                
                print(f"      Protocol: {protocol} | Ports: {from_port}-{to_port}")
                
                # IP ranges
                for ip_range in rule.get('IpRanges', []):
                    cidr = ip_range.get('CidrIp', 'N/A')
                    desc = ip_range.get('Description', '')
                    print(f"        → {cidr} {desc}")
                
                # IPv6 ranges
                for ipv6_range in rule.get('Ipv6Ranges', []):
                    cidr = ipv6_range.get('CidrIpv6', 'N/A')
                    desc = ipv6_range.get('Description', '')
                    print(f"        → {cidr} {desc}")
                
                # Security group references
                for sg_ref in rule.get('UserIdGroupPairs', []):
                    ref_sg_id = sg_ref.get('GroupId', 'N/A')
                    desc = sg_ref.get('Description', '')
                    print(f"        → SG: {ref_sg_id} {desc}")
        else:
            print("      (No outbound rules)")
        
        print("\n" + "-" * 80)
else:
    print("No security groups found attached to instances")




Detailed Security Group Rules:

🔒 Security Group: RemoteAccess (sg-0fff8a116b3c1900a)
   Description: Remote Access SG
   VPC: vpc-0a61263172738c96c

   📥 INBOUND RULES (3):
      Protocol: tcp | Ports: 26-26
        → 207.213.33.0/24 HomeOfficeSSH
        → 23.240.185.115/32 AAA Temp SSH Travel
      Protocol: tcp | Ports: 22-22
        → 23.240.185.115/32 HomeSSH
      Protocol: tcp | Ports: 3389-3389
        → 23.240.185.115/32 HomeOfficeRDP
        → 172.56.240.0/23 AAA MP Class A
        → 32.132.52.0/23 AAA Temp RDP Travel 2

   📤 OUTBOUND RULES (1):
      Protocol: -1 | Ports: All-All
        → 0.0.0.0/0 

--------------------------------------------------------------------------------

🔒 Security Group: SSMVPCEndpoints (sg-00c62ab97c95dd9c0)
   Description: sg for SSMPrivateEndpoint
   VPC: vpc-0a61263172738c96c

   📥 INBOUND RULES (1):
      Protocol: tcp | Ports: 443-443
        → 172.31.0.0/16 vpc ip

   📤 OUTBOUND RULES (2):
      Protocol: -1 | Ports: All-All
        → 0